In [26]:
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup

In [27]:
data = pd.read_csv(r"C:\Users\user\Desktop\DATA BACK UP\Data_sdsouza33\LAMBTON\SEM 3\NLP\project\pse_isr_reddit_comments.csv")

In [28]:
# as per the dataset, doing some pre processing of converting system time to UTC standard DT
data['created_time'] = pd.to_datetime(data['created_time']).dt.tz_convert('UTC')
data.head()

,comment_id,score,self_text,subreddit,created_time
0,k7kxio7,1,k,CombatFootage,2023-11-02 23:49:37+00:00
1,k7kxho7,1,What is he firing? Looks home made? And what i...,CombatFootage,2023-11-02 23:49:25+00:00
2,k7kxhem,1,I mean you did run away from the question firs...,CombatFootage,2023-11-02 23:49:22+00:00
3,k7kxgzg,2,There is a difference between footage showing ...,CombatFootage,2023-11-02 23:49:18+00:00
4,k7kxgne,1,Not feeding you anymore.,CombatFootage,2023-11-02 23:49:14+00:00


In [29]:
data.describe()

,score
count,513557.000000
mean,25.441121
std,164.527875
min,-1372.000000
25%,1.000000
50%,2.000000
75%,9.000000
max,16463.000000


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513557 entries, 0 to 513556
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   comment_id    513557 non-null  object             
 1   score         513557 non-null  int64              
 2   self_text     513555 non-null  object             
 3   subreddit     513557 non-null  object             
 4   created_time  513557 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 19.6+ MB


In [35]:
# cleaning data - removing stopwords, html data, URLs or any special character

# removing URLs
data['self_text'] = data['self_text'].apply(lambda x: re.sub(r'http\S+', '', x) if pd.notnull(x) else x)  #https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
# applied pd.notnull as it gives an error since our data has null value and .apply cant take in non string value

# removing HTML data
data['self_text'] = data['self_text'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text() if pd.notnull(x) else x) #https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string

# removing speciall chars
data['self_text'] = data['self_text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x).lower() if pd.notnull(x) else x)



C:\Users\user\AppData\Local\Temp\ipykernel_20296\1982359991.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data['self_text'] = data['self_text'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text() if pd.notnull(x) else x) #https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string


In [39]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english') )

In [46]:
# tokenization and stop words removal
data['self_text'] = data['self_text'].apply(
    lambda text: ' '.join(
        word.lower() 
        for word in word_tokenize(text) 
        if word.lower() not in stop_words and word.isalnum()
    ) if pd.notnull(text) else text
)

In [8]:
# Question 
#1. Identify the most biased subreddits and understand their stance on the conflict. 
# How do we do this ? What if we check the sentiment of each subreddit and see their sentiment towards palestine/Israel ?
# But that wouldnt solve bias, it just gives me sentiment   
print(data['subreddit'].unique())


# logic 

'''
First we calculcate the sentiment for each comment and then group all the positive and negative sentiment by subreddit groups
Once thats done, we will see how biased the positive and negative sentiments are pos - neg and then check if its more positively biased 
or negatively. 


'''

['CombatFootage' 'IsraelPalestine' 'Palestine' 'AskMiddleEast' 'worldnews'
 'PublicFreakout' 'worldnewsvideo' 'ActualPublicFreakouts'
 'NonCredibleDefense' 'CrazyFuckingVideos' 'IsrealPalestineWar_23'
 'TerrifyingAsFuck' 'AbruptChaos' 'NoahGetTheBoat']
